## Imports

In [635]:
#Rawan's Push

In [636]:
import pandas as pd
import os
import numpy as np


## Functions 

### Find Each Table's primary key

In [637]:
def pk_finder(name, df):
    for i in range(len(df.columns)):
        col = df.iloc[:,i]
        #if(col.size == col.drop_duplicates().size):
        if(col.nunique() == col.size):
            pk[name] = df.columns[i] 
            break
        

### Find Each Table's foreign key


In [638]:
def fk_finder(name, df):
    for i in range(len(df.columns)):
        col_name = df.columns[i]
        col = df.iloc[:,i]
        if(list(df.columns)[i] not in pk):
            flag = True
            for j in pk: #loops on the key j = table names
                if j != name:
                    primary_key_column = All_dfs[j][(pk[j])]
                    if primary_key_column.dtype == col.dtype:
                        status = col.isin(primary_key_column)
                        if status[status== False].size == 0:
                            fk[name][col_name] = j 
                            ref_in[j].add(name)
                            break

### Assign Tables to nodes or edges

In [639]:
def nodes_edges(fk):
    for f in fk:
        if len(fk[f]) == 2:
            inner_dict = list(fk[f])
            edges[f] = fk[f]
        elif len(fk[f]) == 1:
            properties[f] = fk[f]
        else:
            nodes[f] = All_dfs[f]
#         else:
#             if len(fk[f]) == 1:
#                 properties[f] = fk[f]
            
#             nodes[f] = All_dfs[f]
    # for f in fk:
    #     if((f not in edges) ):
    #         nodes[f] = All_dfs[f]
    #         #nodes.append(f)
        

### Create Nodes Table

In [640]:
def add_nodes(nodes, nodesTable):
    for n in range(len(nodes)):
        for index, row in nodes[list(nodes.keys())[n]].iterrows():
            column_names = list(nodes[list(nodes.keys())[n]].columns) #get column names
            att = {}
            for i in range(1, len(column_names)):
                att[column_names[i]] = nodes[list(nodes.keys())[n]].iloc[index, i]
            newRow = [{'Label': list(nodes.keys())[n] , 'ID': nodes[list(nodes.keys())[n]].iloc[index,0] , 'Attributes': att}]
            tmp = pd.DataFrame(newRow)
            nodesTable = pd.concat([nodesTable, tmp], ignore_index=True)
    return nodesTable


### Create Edges Table

In [641]:
def add_edges(edges,edgesTable,pk,fk,ref_in):
    for e in edges:
        df = All_dfs[e] #df of the cur edge
        for r in range(len(df)):
            label = list(edges[e].keys())
            from_id = nodesTable[(nodesTable["Label"] == edges[e][label[0]]) & (df.loc[r,label[0]] == nodesTable["ID"])]
            from_id = from_id.index[0]
            to_id = nodesTable[(nodesTable["Label"] == edges[e][label[1]]) & (df.loc[r,label[1]] == nodesTable["ID"])]
            to_id = to_id.index[0]
            pk_col = pk[e]
#             primary_key = df[pk_col].iloc[r]
            primary_key = df.loc[r,pk_col]
            att = convert_prop(ref_in,e,primary_key,fk)
            newRow = [{'From_Node_ID': from_id, 'To_Node_ID': to_id, 'order/service' : att}]
            tmp = pd.DataFrame(newRow)
            edgesTable = pd.concat([edgesTable, tmp], ignore_index=True)
    return edgesTable

### Properties of the Edge

In [642]:
def convert_prop(ref_in,edge_key, pk_value,fk):
    for referenced_table_name in ref_in[edge_key]:
        referenced_table = All_dfs[referenced_table_name]
        fk_of_ref_table = fk[referenced_table_name]
        for foreign_key in fk_of_ref_table:
            if fk_of_ref_table[foreign_key] == edge_key:
#                 referenced_table[referenced_table[foreign_key]== id]
                all_occurances_df = referenced_table[referenced_table[foreign_key]== pk_value].drop([foreign_key],axis=1)
                return all_occurances_df.to_dict('records')
                

## Global Variables

In [643]:
# {Table name: df}
All_dfs = {}
# {Table name: Table's Primary key}
pk = {}
# {Table name: {Column Name: Referenced Table}}
fk = {}
# {Table name: set(Tables that references the table)}
ref_in = {}
# {Table name: df}
nodes = {}
# {Table name: {Column Name: Referenced Table}}
edges = {}
# _
properties = {}
# Nodes_df
nodesTable = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
# Edges_df
edgesTable = pd.DataFrame(columns = ['From_Node_ID', 'To_Node_ID', 'order/service'])


## Reading Data Set

 creating a dictionary where
 key: first word of the table name 
 value: df 

In [644]:
import os
path_of_the_directory = './DataSet/'
ext = ('.csv')
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        print(file) 
        temp = (file.replace("_"," ").replace("."," ").split(" ")[0].lower())
        All_dfs[temp] = pd.read_csv(path_of_the_directory+file)
    else:
        continue

InternalTransactions.csv
InternalShipments_data.csv
facilities_data.csv
ExternalShipments_data.csv
InternalServices_data.csv
warehouses_data.csv
InternalOrders_data.csv
customer_data.csv
Products_data.csv
Supplier_data.csv
ExternalTransactions.csv
ExternalOrders_data.csv
Manufacturing_data.csv
ExternalServices_data.csv
Retailer_data.csv


In [645]:
All_dfs["externalshipments"]

,ExtShip_id,factoryIds/retailerIds,idsTo,from_to_where
0,2108,63898,12666,RC
1,1456,99762,15724,SC
2,6720,26336,83792,RC
3,5089,97142,91846,SC
4,7708,26533,39892,SC
...,...,...,...,...
95,7218,97142,29456,SC
96,3429,46204,70649,SC
97,3852,12318,82485,RC
98,8524,61580,87252,RC


## Adding Missing Edges

In [646]:
c = 9995
for r in range(len(All_dfs["manufacturing"])):
    diff_supp = All_dfs["manufacturing"].loc[r,"Different_suppliers"]
    diff_supp = diff_supp.split(",")
    diff_supp[0] = diff_supp[0].strip("(")
    diff_supp[len(diff_supp)-1] = diff_supp[len(diff_supp)-1].strip(")")
    factory = All_dfs["manufacturing"].loc[r,"Factory_id"]
    x = All_dfs["internalshipments"].listSuppIds
    y = All_dfs["externalshipments"]["factoryIds/retailerIds"]
    i = ""
    for s in range(len(diff_supp)):
        if diff_supp[s] == factory:
            print(factory)
        if (diff_supp[s] not in x) & (diff_supp[s] not in y):
                if factory in All_dfs["supplier"].supp_id:
                    i = "SS"

        new_row = [{"IntShip_id":c, "listSuppIds":diff_supp[s], "factoryIds": factory, "from_to_where": i}] 
        df = pd.DataFrame(new_row)
        c = c+1
        All_dfs["internalshipments"] = pd.concat([All_dfs["internalshipments"], df], ignore_index=True)
        

In [647]:
All_dfs["internalshipments"]

,IntShip_id,listSuppIds,factoryIds,from_to_where
0,7281,90942,52160,SS
1,5897,36243,67361,SR
2,3477,87916,30692,SS
3,8796,90236,26533,SS
4,2360,89467,46954,SR
...,...,...,...,...
611,10506,48467,46070,
612,10507,70468,18952,
613,10508,87916,18952,
614,10509,48580,18952,


In [648]:
# serv_dfs = [All_dfs["internalservices"],All_dfs["externalservices"]]
# tran_dfs = [All_dfs["internaltransactions"],All_dfs["externaltransactions"]]
# for df in range(0,2):
#     for r in range(len(serv_dfs[df])):
#         tran_id = serv_dfs.iloc[r,1]
        

### Splitting Shipments tables

In [649]:
SRIntShip = All_dfs["internalshipments"].query('from_to_where == "SR"')
SRIntShip = SRIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SRIntShip
RCExtShip = All_dfs["externalshipments"].query('from_to_where == "RC"')
RCExtShip = RCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#RCExtShip
SSIntShip = All_dfs["internalshipments"].query('from_to_where == "SS"')
SSIntShip = SSIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SSIntShip
SCExtShip = All_dfs["externalshipments"].query('from_to_where == "SC"')
SCExtShip = SCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SCExtShip
All_dfs["RCExtShip".lower()] = RCExtShip
All_dfs["SCExtShip".lower()] = SCExtShip
All_dfs["SRIntShip".lower()] = SRIntShip
All_dfs["SSIntShip".lower()] = SSIntShip

### Splitting Orders Tables

In [650]:
filter_list = All_dfs["rcextship"]["ExtShip_id"]
All_dfs["rcextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["scextship"]["ExtShip_id"]
All_dfs["scextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["srintship"]["IntShip_id"]
All_dfs["srintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["ssintship"]["IntShip_id"]
All_dfs["ssintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)

### Removing Redundant Tables

In [651]:
All_dfs.pop("internalshipments")
All_dfs.pop("externalshipments")
All_dfs.pop("externalorders")
All_dfs.pop("internalorders")
All_dfs.keys()

dict_keys(['internaltransactions', 'facilities', 'internalservices', 'warehouses', 'customer', 'products', 'supplier', 'externaltransactions', 'manufacturing', 'externalservices', 'retailer', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

### Initialize fk and ref_in

In [652]:
table_name = list(All_dfs.keys())
for table_name in All_dfs:
    fk[table_name] = {}
    ref_in[table_name] = set()

### Get Primary key for each table

In [653]:
for t in All_dfs:
    pk_finder(t,All_dfs[t])
pk

{'internaltransactions': 'IntTran_id',
 'facilities': 'fac_id',
 'internalservices': 'IntServ_id',
 'warehouses': 'warehouse_id',
 'customer': 'cust_id',
 'products': 'prod_id',
 'supplier': 'supp_id',
 'externaltransactions': 'ExtTran_id',
 'manufacturing': 'Manf_id',
 'externalservices': 'ExtServ_id',
 'retailer': 'retailer_id',
 'rcextship': 'ExtShip_id',
 'scextship': 'ExtShip_id',
 'srintship': 'IntShip_id',
 'ssintship': 'IntShip_id',
 'rcextorders': 'ExtOrders_id',
 'scextorders': 'ExtOrders_id',
 'srintorders': 'IntOrders_id',
 'ssintorders': 'IntOrders_id'}

### Get Foreing key for each table

In [654]:
for t in All_dfs:
    fk_finder(t,All_dfs[t])
fk

{'internaltransactions': {'CompFrom': 'supplier', 'Compto': 'supplier'},
 'facilities': {'supplier_id': 'supplier'},
 'internalservices': {'IntTrans_id': 'internaltransactions'},
 'warehouses': {},
 'customer': {},
 'products': {},
 'supplier': {},
 'externaltransactions': {'CompFrom': 'supplier', 'Custto': 'customer'},
 'manufacturing': {'Product_id': 'products', 'Factory_id': 'supplier'},
 'externalservices': {'ExtTrans_id': 'externaltransactions'},
 'retailer': {},
 'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'scextship': {'factoryIds/retailerIds': 'supplier', 'idsTo': 'customer'},
 'srintship': {'factoryIds': 'retailer'},
 'ssintship': {'factoryIds': 'supplier'},
 'rcextorders': {'ExtShip_id': 'rcextship'},
 'scextorders': {'ExtShip_id': 'scextship'},
 'srintorders': {'IntShip_id': 'srintship'},
 'ssintorders': {'IntShip_id': 'ssintship'}}

In [655]:
ref_in

{'internaltransactions': {'internalservices'},
 'facilities': set(),
 'internalservices': set(),
 'warehouses': set(),
 'customer': {'externaltransactions', 'rcextship', 'scextship'},
 'products': {'manufacturing'},
 'supplier': {'externaltransactions',
  'facilities',
  'internaltransactions',
  'manufacturing',
  'scextship',
  'ssintship'},
 'externaltransactions': {'externalservices'},
 'manufacturing': set(),
 'externalservices': set(),
 'retailer': {'rcextship', 'srintship'},
 'rcextship': {'rcextorders'},
 'scextship': {'scextorders'},
 'srintship': {'srintorders'},
 'ssintship': {'ssintorders'},
 'rcextorders': set(),
 'scextorders': set(),
 'srintorders': set(),
 'ssintorders': set()}

### Determine nodes and edges

In [656]:
nodes_edges(fk)

In [657]:
nodes.keys()

dict_keys(['warehouses', 'customer', 'products', 'supplier', 'retailer'])

In [658]:
edges.keys()

dict_keys(['internaltransactions', 'externaltransactions', 'manufacturing', 'rcextship', 'scextship'])

In [659]:
properties.keys()

dict_keys(['facilities', 'internalservices', 'externalservices', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

### Create NodesTable

In [660]:
nodesTable = add_nodes(nodes,nodesTable)
nodesTable

,Label,ID,Attributes
0,warehouses,3144,"{'country': 'Jamaica', 'city_name': 'Saint And..."
1,warehouses,5657,"{'country': 'United Arab Emirates', 'city_name..."
2,warehouses,8630,"{'country': 'Mexico', 'city_name': 'Sonora', '..."
3,warehouses,6178,"{'country': 'Senegal', 'city_name': 'Louga', '..."
4,warehouses,6560,"{'country': 'Aruba', 'city_name': nan, 'capaci..."
...,...,...,...
1195,retailer,86308,"{'country': 'Ukraine', 'city_name': 'Chernihiv..."
1196,retailer,18464,"{'country': 'Haiti', 'city_name': 'Grandans', ..."
1197,retailer,57039,{'country': 'Lao People's Democratic Republic'...
1198,retailer,33250,"{'country': 'South Sudan', 'city_name': 'Lakes..."


### Create Edges Table

In [661]:
edgesTable = add_edges(edges, edgesTable,pk,fk,ref_in)
edgesTable

,From_Node_ID,To_Node_ID,order/service
0,869,909,"[{'IntServ_id': 2070, 'placed_when': '2002-01-..."
1,924,1043,"[{'IntServ_id': 1165, 'placed_when': '2000-08-..."
2,1003,996,"[{'IntServ_id': 4897, 'placed_when': '2009-04-..."
3,953,1009,[]
4,847,1012,[]
...,...,...,...
395,883,239,[]
396,908,129,"[{'ExtOrders_id': 5650, 'quantity': 268, 'plac..."
397,844,167,"[{'ExtOrders_id': 8940, 'quantity': 706, 'plac..."
398,756,125,"[{'ExtOrders_id': 9052, 'quantity': 217, 'plac..."


In [662]:
All_dfs["facilities"].fac_id.value_counts()

694420    1
894911    1
990810    1
629057    1
698102    1
         ..
643667    1
434354    1
419150    1
542930    1
186890    1
Name: fac_id, Length: 2000, dtype: int64

In [663]:
properties.keys()

dict_keys(['facilities', 'internalservices', 'externalservices', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

In [664]:
for property in range(len(list(properties.keys()))):
  table_name = 'facilities'
  if(list(properties.keys())[property] == table_name):
    referencedTableName = properties[table_name][list(properties[table_name].keys())[0]]
    if(referencedTableName in nodes.keys()):
      print("Referenced Table is Node")
    elif(referencedTableName in edges.keys()):
      print("Referenced Table is Edge")
    referencedColName = list(properties[table_name].keys())[0]
    fac_df = All_dfs[list(properties.keys())[property]]
    for index in range(fac_df.shape[0]):
      fac = fac_df.iloc[index]
      fromId = fac[pk[list(properties.keys())[property]]]
      toId = fac[referencedColName]
      print(fromId)
      print(toId)
      print("---------")
      # print(All_dfs[list(properties.keys())[property]])
    

Referenced Table is Node
694420
81114
---------
678955
81114
---------
719035
81114
---------
240432
81114
---------
385384
81114
---------
661839
81114
---------
263477
81114
---------
144639
81114
---------
858404
81114
---------
991077
81114
---------
915035
68177
---------
905177
68177
---------
701418
68177
---------
952688
68177
---------
161033
68177
---------
480583
68177
---------
909341
68177
---------
480533
68177
---------
699500
68177
---------
404028
68177
---------
957998
70869
---------
907715
70869
---------
790711
70869
---------
282126
70869
---------
399770
70869
---------
898663
70869
---------
577085
70869
---------
559237
70869
---------
711270
70869
---------
869533
70869
---------
739435
99101
---------
726402
99101
---------
950184
99101
---------
698173
99101
---------
268235
99101
---------
310760
99101
---------
159199
99101
---------
872449
99101
---------
170447
99101
---------
854146
99101
---------
223443
58047
---------
264966
58047
---------
114492
58

In [665]:
All_dfs.keys()

dict_keys(['internaltransactions', 'facilities', 'internalservices', 'warehouses', 'customer', 'products', 'supplier', 'externaltransactions', 'manufacturing', 'externalservices', 'retailer', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

In [666]:
nodes.keys()

dict_keys(['warehouses', 'customer', 'products', 'supplier', 'retailer'])

In [667]:
edges.keys()

dict_keys(['internaltransactions', 'externaltransactions', 'manufacturing', 'rcextship', 'scextship'])

In [668]:
properties.keys()

dict_keys(['facilities', 'internalservices', 'externalservices', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

In [669]:
fk

{'internaltransactions': {'CompFrom': 'supplier', 'Compto': 'supplier'},
 'facilities': {'supplier_id': 'supplier'},
 'internalservices': {'IntTrans_id': 'internaltransactions'},
 'warehouses': {},
 'customer': {},
 'products': {},
 'supplier': {},
 'externaltransactions': {'CompFrom': 'supplier', 'Custto': 'customer'},
 'manufacturing': {'Product_id': 'products', 'Factory_id': 'supplier'},
 'externalservices': {'ExtTrans_id': 'externaltransactions'},
 'retailer': {},
 'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'scextship': {'factoryIds/retailerIds': 'supplier', 'idsTo': 'customer'},
 'srintship': {'factoryIds': 'retailer'},
 'ssintship': {'factoryIds': 'supplier'},
 'rcextorders': {'ExtShip_id': 'rcextship'},
 'scextorders': {'ExtShip_id': 'scextship'},
 'srintorders': {'IntShip_id': 'srintship'},
 'ssintorders': {'IntShip_id': 'ssintship'}}

In [670]:
pk

{'internaltransactions': 'IntTran_id',
 'facilities': 'fac_id',
 'internalservices': 'IntServ_id',
 'warehouses': 'warehouse_id',
 'customer': 'cust_id',
 'products': 'prod_id',
 'supplier': 'supp_id',
 'externaltransactions': 'ExtTran_id',
 'manufacturing': 'Manf_id',
 'externalservices': 'ExtServ_id',
 'retailer': 'retailer_id',
 'rcextship': 'ExtShip_id',
 'scextship': 'ExtShip_id',
 'srintship': 'IntShip_id',
 'ssintship': 'IntShip_id',
 'rcextorders': 'ExtOrders_id',
 'scextorders': 'ExtOrders_id',
 'srintorders': 'IntOrders_id',
 'ssintorders': 'IntOrders_id'}

In [671]:
All_dfs["rcextship"]

,ExtShip_id,factoryIds/retailerIds,idsTo
0,2108,63898,12666
1,6720,26336,83792
2,5965,31857,16956
3,4536,67158,45986
4,2575,26654,64367
5,1281,90345,84885
6,4492,91089,50014
7,8644,22049,90435
8,6650,94027,21113
9,1139,45151,43022


In [672]:
x = All_dfs["rcextship"]["factoryIds/retailerIds"].isin(All_dfs["retailer"].retailer_id)
x.value_counts()


True    51
Name: factoryIds/retailerIds, dtype: int64

In [673]:
x = All_dfs["rcextship"]["factoryIds/retailerIds"].isin(All_dfs["retailer"].retailer_id)
x.value_counts()

True    51
Name: factoryIds/retailerIds, dtype: int64

In [674]:
 All_dfs["rcextship"]["idsTo"].value_counts()

42770    3
22228    2
45986    2
21113    2
76264    2
48071    2
18779    1
14142    1
82092    1
80996    1
87421    1
32819    1
62831    1
21478    1
13248    1
31548    1
50653    1
47008    1
96784    1
79671    1
65924    1
12717    1
82485    1
14574    1
12666    1
32489    1
96548    1
16956    1
64367    1
84885    1
50014    1
90435    1
43022    1
81118    1
66152    1
83792    1
85709    1
60542    1
63479    1
80020    1
41344    1
24998    1
72646    1
87252    1
Name: idsTo, dtype: int64